In [1]:
import os
import glob
import cv2
import random
import pandas as pd
from skimage import io
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import h5py

# Network building stuff
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

In [2]:
import sys
sys.path.append("/home2/sdokania/all_projects/project-noisypixel/")

In [3]:
from src.models import *
from src.dataset.dataloader import OccupancyNetDatasetHDF
from src.trainer import ONetLit
from src.utils import Config, count_parameters

In [4]:
config = Config()
config.data_root = "/ssd_scratch/cvit/sdokania/processed_data/hdf_data/"
config.batch_size = 32

In [5]:

vars(config)

{'c_dim': 128,
 'h_dim': 128,
 'p_dim': 3,
 'data_root': '/ssd_scratch/cvit/sdokania/processed_data/hdf_data/',
 'batch_size': 32,
 'output_dir': '/home2/sdokania/all_projects/occ_artifacts/',
 'exp_name': 'initial',
 'encoder': 'efficientnet-b0',
 'decoder': 'decoder-cbn',
 'exp_path': '/home2/sdokania/all_projects/occ_artifacts/initial',
 'lr': 0.0003}

In [9]:
onet = ONetLit(config)

Loaded pretrained weights for efficientnet-b0


In [10]:
onet(torch.randn(1, 3, 224, 224), torch.randn(1, 1024, 3)).shape

torch.Size([1, 1024])

In [13]:
# Define the trainer object
trainer = pl.Trainer(
    gpus=1,
    # auto_scale_batch_size='binsearch',
    logger=logger,
    min_epochs=1,
    max_epochs=1,
    default_root_dir=config.output_dir,
    log_every_n_steps=10,
    progress_bar_refresh_rate=5,
    # precision=16,
    # stochastic_weight_avg=True,
    # track_grad_norm=2,
    callbacks=[checkpoint_callback],
    check_val_every_n_epoch=1,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [14]:
# Start training
trainer.fit(onet)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type      | Params
-----------------------------------
0 | net  | OccNetImg | 4.7 M 
-----------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.802    Total estimated model params size (MB)
/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [7]:
vd = OccupancyNetDatasetHDF(config.data_root, mode="val")
vdl = torch.utils.data.DataLoader(vd, batch_size=100, shuffle=False, num_workers=8)

In [8]:
for ix in vdl:
    print(ix[0].shape, ix[1].shape, ix[2].shape)

torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100, 3, 137, 137]) torch.Size([100, 1024, 3]) torch.Size([100, 1024])
torch.Size([100,

In [42]:
len(vdl)

44

In [13]:
fname = "/ssd_scratch/cvit/sdokania/hdf_data/hdf_data/04256520_bdfcf2086fafb0fec8a04932b17782af.h5"
hf = h5py.File(fname, 'r')

In [15]:
hf.keys()

<KeysViewHDF5 ['camera', 'images']>